In [1]:
%pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.


In [5]:
import speech_recognition as sr

In [6]:
r = sr.Recognizer()
test_file = sr.AudioFile('test_file1.wav')
with test_file as source:
    audio = r.record(source)

In [11]:
out = r.recognize_google(audio)
print(out)

painted the sockets in the Wall don't green the child crawled into the dense grass Brides fail we're honest man work trampled the spark the flames will spread the hilt of the sword was carved with fine designs around hole was drilled through the thin board footprints showed the path he took up the beach she was waiting at my front 10 event near the edge brought in fresh air prod the Old mule with a crooked stick
